In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
dataset = 'IllusionAnimals'
file_name = 'CLIP_IllusionAnimals_test'

df = pd.read_csv(f'./{file_name}.csv')

In [2]:
df

,image_name,Pprompt,Nprompt,illusion_strength,label,raw_answer,ill_answer,illless_answer,ill_filter_answer,illless_filter_answer
0,IllusionAnimals_1,A raging river flowing through a dense jungle ...,low quality,2.5,cat,cat,illusion animal elephant,no illusion animal,illusion animal cat,illusion animal deer
1,IllusionAnimals_2,A starry night sky over a tranquil lake,low quality,2.5,cat,cat,illusion animal deer,illusion animal deer,illusion animal cat,no illusion animal
2,IllusionAnimals_3,Jaguar (Panthera Onca) patrolling South Americ...,low quality,2.5,cat,cat,illusion animal deer,no illusion animal,illusion animal cat,illusion animal cat
3,IllusionAnimals_4,"Flowing lava illuminates cavern walls, ancient...",low quality,2.5,cat,cat,illusion animal elephant,no illusion animal,illusion animal cat,no illusion animal
4,IllusionAnimals_5,"Dense forest canopy, sunlight filters through,...",low quality,2.5,cat,cat,illusion animal deer,illusion animal deer,illusion animal cat,illusion animal elephant
...,...,...,...,...,...,...,...,...,...,...
995,IllusionAnimals_996,Saharan sandstorm turning daytime into darkness,low quality,2.5,rooster,rooster,illusion animal rooster,illusion animal snake,illusion animal rooster,illusion animal snake
996,IllusionAnimals_997,Urban city with skyscrapers and traffic,low quality,2.5,rooster,rooster,illusion animal elephant,illusion animal snake,illusion animal rooster,no illusion animal
997,IllusionAnimals_998,A serene meadow with wildflowers and butterflies,low quality,2.5,rooster,rooster,illusion animal horse,no illusion animal,illusion animal rooster,illusion animal deer
998,IllusionAnimals_999,Elk bugling in crisp autumn air,low quality,2.5,rooster,rooster,illusion animal deer,illusion animal deer,illusion animal rooster,illusion animal deer


In [3]:
import torch
import numpy as np
import random

# Make reproducible code
GLOBAL_SEED = 10

np.random.seed(GLOBAL_SEED)
random.seed(GLOBAL_SEED)
torch.manual_seed(GLOBAL_SEED)
torch.use_deterministic_algorithms(True)
%env CUBLAS_WORKSPACE_CONFIG=:4096:8

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

illusionless_indexes = random.sample(range(len(df)), int(len(df)*0.1))

env: CUBLAS_WORKSPACE_CONFIG=:4096:8


# **Raw**

In [4]:
y_true = []
y_pred = []

In [5]:
y_true = df['label'].apply(lambda x: str(x)).tolist()
y_pred = df[f'raw_answer'].apply(lambda x: x.strip().lower()).tolist()

In [6]:
y_pred_ = [item.split()[-1] for item in y_pred]

In [7]:
accuracy = accuracy_score(y_true, y_pred_)
precision = precision_score(y_true, y_pred_, average='macro')
recall = recall_score(y_true, y_pred_, average='macro')
f1 = f1_score(y_true, y_pred_, average='macro')

print("Accuracy:", round(accuracy, 4))
print("Precision:", round(precision, 4))
print("Recall:", round(recall, 4))
print("F1 score:", round(f1, 4))

Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1 score: 1.0


# **Illusion**

In [8]:
y_true = []
y_pred = []

In [9]:
y_pred_illless = [df.at[row, 'illless_answer'].split()[-1] for row in illusionless_indexes]
y_true_illless = ['animal' for i in range(len(y_pred_illless))]

In [10]:
y_true_ill = df['label'].apply(lambda x: str(x)).tolist()
y_pred_ill = df[f'ill_answer'].apply(lambda x: x.strip().lower().split()[-1]).tolist()

In [11]:
y_true = y_true_ill + y_true_illless
y_pred = y_pred_ill + y_pred_illless

In [12]:
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='macro')
recall = recall_score(y_true, y_pred, average='macro')
f1 = f1_score(y_true, y_pred, average='macro')

print("Accuracy:", round(accuracy, 4))
print("Precision:", round(precision, 4))
print("Recall:", round(recall, 4))
print("F1 score:", round(f1, 4))

Accuracy: 0.4264
Precision: 0.462
Recall: 0.4264
F1 score: 0.3956


# **Filtered**

In [13]:
y_true = []
y_pred = []

In [14]:
y_pred_illless = [df.at[row, 'illless_filter_answer'].split()[-1] for row in illusionless_indexes]
y_true_illless = ['animal' for i in range(len(y_pred_illless))]

In [15]:
y_true_ill = df['label'].apply(lambda x: str(x)).tolist()
y_pred_ill = df[f'ill_filter_answer'].apply(lambda x: x.strip().lower().split()[-1]).tolist()

In [16]:
y_true = y_true_ill + y_true_illless
y_pred = y_pred_ill + y_pred_illless

In [17]:
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='macro')
recall = recall_score(y_true, y_pred, average='macro')
f1 = f1_score(y_true, y_pred, average='macro')

print("Accuracy:", round(accuracy, 4))
print("Precision:", round(precision, 4))
print("Recall:", round(recall, 4))
print("F1 score:", round(f1, 4))

Accuracy: 0.8545
Precision: 0.8637
Recall: 0.8545
F1 score: 0.8519
